# Logical Transduction

Agentics objects are capable of performing logical transduction between their states by using the provied LLMs. The logical transduction operator << can be applied between any two agentics of any type.

Transduction is done between a source and a target AG when connected by the << operator.

The follwing example transduces a Question into an Answer.

In [1]:
! uv pip install agentics-py


import os
import sys

from dotenv import find_dotenv, load_dotenv

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)

if IN_COLAB:
    CURRENT_PATH = "/content/drive/MyDrive/"
    from google.colab import drive
    drive.mount("/content/drive")
    load_dotenv("/content/drive/MyDrive/.env")
else:
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = input("Enter your GEMINI_API_KEY:")

Using Python 3.12.9 environment at: /Users/gliozzo/Code/agentics911/agentics/.venv
Audited 1 package in 60ms
In Colab: False
In Colab: False


In [ ]:
from agentics import AG
from pydantic import BaseModel
from typing import Optional



## Define target and source types
class Answer(BaseModel):
    answer: Optional[str] = None
    justification: Optional[str] = None
    confidence: Optional[float] = None


class Question(BaseModel):
    question: Optional[str] = None


## Instantiate the source AG with a question
source = AG(atype=Question,
            llm=AG.get_llm_provider(), ## You can choose between "openai" (i.e. get_llm_provider("openai")), "watsonx", "gemini", "vllm_crewai"
                                    ## set verbose to true to see the internal agents log. This is optional.
            states=[Question(question="What is the capital of Italy?")])

## Instantiate the target AG with a target type. No instances are needed for zero shot transduction
target = AG(
    atype=Answer,  ## You can choose between "openai", "watsonx", "gemini", "vllm_crewai"
    verbose_agent=True,
)  ## set verbose to true to see the internal agents log

# Execute logical transduction by using the << operator between source and target AG
answer = await (
    target << source
)  ## Note that << operator is asyncronus and the results should be awaited

# Print the results of the transduction
print(
    answer.pretty_print()
)  ## Note that confidence is a float number, while other fields are strings

2025-09-23 15:35:58.682 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0f916d0c-36d0-40b3-aa44-660c455f4d76                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Task Executor                                                                                           │
│                                                                                                                 │
│  Task:                                                                                                          │
│  Your task is to transduce a source Pydantic Object into the specified Output type. Generate only slots that    │
│  are logically deduced from the input information, otherwise live then null.                                    │
│                                                                                                                 │
│  Read carefully the following instructions for executing your task:                                             │
│  Generate an object of the specified type from the following input. SOURCE:                                     │
│  {"question": "What is the capital of Italy?"}                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Task Executor                                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "answer": "Rome",                                                                                            │
│    "justification": "Rome is widely recognized as the capital of Italy.",                                       │
│    "confidence": 0.99                                                                                           │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b00638c4-92e1-4fae-8007-9c1c20214e14                                                                     │
│  Agent: Task Executor                                                                                           │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 0f916d0c-36d0-40b3-aa44-660c455f4d76                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: {                                                                                                │
│    "answer": "Rome",                                                                                            │
│    "justification": "Rome is widely recognized as the capital of Italy.",                                       │
│    "confidence": 0.99                                                                                           │
│  }                                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

IndexError: list index out of range

In agentics, lists of strings can be used as sources instead of AG. Those are provided as input for the transduction.

In [4]:
answer = await (AG(atype=Answer) << ["Where is Paris?"])
print(answer.pretty_print())

2025-09-23 15:36:27.313 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


Atype : <class '__main__.Answer'>
answer: Paris is located in France, in the northern-central part of the country.
justification: "Paris is widely recognized as the capital of France. Geographically,\
  \ it is situated in the \xCEle-de-France region, which is in the northern-central\
  \ part of France."
confidence: 0.99




## Asyncronous Transduction

When the source AG has more than one state, the << operator perform asyncronous transfuction for each state to the target type. Transductions are executed in batches.

In [5]:
target = AG(
    atype=Answer,
    verbose_transduction=True,  # Set to verbose to see transduction timings and other logs
    transduction_logs_path="/tmp/answers.jsonl",
)  # Optionally write longs of transductions on the specified path
questions = [
    "Where is Paris?",
    "Who is Alberto Sordi",
    "When will climate change be irreverible?",
    "Who is the best Jeopardy player?",
]

answers = await (target << questions)

print(answer.pretty_print())

answers = await (target << questions)

2025-09-23 15:36:34.403 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


Output()

Output()

Atype : <class '__main__.Answer'>
answer: Paris is located in France, in the northern-central part of the country.
justification: "Paris is widely recognized as the capital of France. Geographically,\
  \ it is situated in the \xCEle-de-France region, which is in the northern-central\
  \ part of France."
confidence: 0.99




## Self Transduction

Self transduction is a method of AGs that enables async execution of transductions between slots of the same object.

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional
from pathlib import Path


## Define the Pydantic type
class Movie(BaseModel):
    movie_name: Optional[str] = (
        None  ## Note that fields name should match the column name in the input csv
    )
    genre: Optional[str] = None
    description: Optional[str] = None
    tweet: Optional[str] = Field(
        None, description="Generate a Tweet to advertise the movie"
    )


base = Path(CURRENT_PATH)
movies = AG.from_csv(
    base / "data/movies.csv", atype=Movie, max_rows=20
)  ## Load the input data from a csv file
movies.verbose_transduction = True
movies.llm = AG.get_llm_provider(
    "watsonx"
)  ## You can choose between "openai", "watsonx", "gemini", "vllm_crewai"

movies_with_tweets = await movies.self_transduction(
    ["movie_name", "genre", "description"],  ## source fields
    ["tweet"],  ## target fields
    ## Note that instruction are only needed when the  relation between source and target type
    # is not innediately clear and need to be further specified or disambiguated.
    instructions="Generate a tweet to advertise the release of the input movie",
)

print(movies_with_tweets.pretty_print())

2025-09-23 15:36:52.308 | DEBUG    | agentics.core.agentics:from_csv:260 - Importing Agentics of type Movie from CSV data/movies.csv
2025-09-23 15:36:52.310 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


NameError: name 'get_llm_provider' is not defined

As an alternative, self transduction can be encoded using logical transduction algebra which uses the AG() notation to rebind the original data into AGs of the requested subtypes. Learn more about atype manipulation in agentics [here](link)

In [ ]:
movies = AG.from_csv( base / "data/movies.csv", atype=Movie, max_rows=20)
tweets = await (AG(atype=movies("tweet").atype,
                   instructions="Generate a tweet to advertise the release of the input movie")
                << movies("movie_name","genre","description" ))
print(tweets.pretty_print())    ## Note that differently from self transduction the output tweets
                                ##has only the tweet field, whereas self transduction preserves
                                ## the original source data

2025-09-11 11:26:45.851 | DEBUG    | agentics.core.agentics:from_csv:312 - Importing Agentics of type Movie from CSV data/movies.csv
2025-09-11 11:26:45.854 | DEBUG    | agentics.core.agentics:__lshift__:518 - Executing task: Generate a tweet to advertise the release of the input movie
20 states will be transduced
2025-09-11 11:26:45.855 | DEBUG    | agentics.core.agentics:__lshift__:612 - transducer class: <class 'agentics.abstractions.pydantic_transducer.PydanticTransducerCrewAI'>
2025-09-11 11:26:50.763 | DEBUG    | agentics.core.agentics:__lshift__:648 - Processed 20 states in 4.907993793487549 seconds
2025-09-11 11:26:50.764 | DEBUG    | agentics.core.agentics:__lshift__:700 - 20 states processed in 0.24539968967437745 seconds average per state ...


Atype : <class 'agentics.core.agentics.tweet'>
tweet: 'Get ready to experience hope and redemption! ''The Shawshank Redemption'',
  a gripping Drama and Crime movie, is now out! Imprisoned for a crime he didn''t
  commit, Andy Dufresne''s story of integrity and hope will leave you inspired. Don''t
  miss it! #TheShawshankRedemption #NewRelease'

tweet: 'Get ready for a cinematic masterpiece! ''The Godfather'', a gripping Drama
  and Crime film, is now out! Spanning 1945 to 1955, it chronicles the Italian-American
  Corleone crime family''s bloody revenge saga. Don''t miss it! #TheGodfather #Drama
  #Crime'

tweet: 'Get ready for a cinematic masterpiece! ''The Godfather Part II'' is now out!
  This Drama, Crime film follows the Corleone crime family''s saga as they expand
  their business into Las Vegas, Hollywood, and Cuba. Don''t miss out on the thrilling
  story of the Corleone family''s rise to power! #TheGodfatherPartII #Drama #Crime'

tweet: 'Get ready to witness a powerful true s

## Few Shots Transduction

Few shots examples can be provided for transduction by adding instances of the target instances in correspondance to their sources . Those will be used by the LLM to infer by analogy all the Null instances of the target type.


In [15]:
movies = AG.from_csv(base / "data/movies.csv", max_rows=20, atype=Movie)

print(movies.atype)

2025-09-23 16:16:24.395 | DEBUG    | agentics.core.agentics:from_csv:260 - Importing Agentics of type Movie from CSV data/movies.csv
2025-09-23 16:16:24.397 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


<class '__main__.Movie'>


In [ ]:
movies = AG.from_csv(base / "data/movies.csv", max_rows=20)
## Note that obly the first 9 movies have categories
for i, movie in enumerate(movies):
    print(f"{i}: {movie.genre}")
## predicting new genre from given examples
all_genres = await (movies("genre") << movies("movie_name", "description"))
print(all_genres.pretty_print())
## few shots examples can also be used in self transfuction
movies_with_genre = await movies.self_transduction(
    ["movie_name", "description"], ["genre"]
)
print(movies_with_genre.pretty_print())

2025-09-23 15:37:58.883 | DEBUG    | agentics.core.llm_connections:get_llm_provider:30 - Available LLM providers: ['watsonx', 'gemini', 'openai']. None specified, defaulting to 'watsonx'


Output()

0: Drama, Crime
1: Drama, Crime
2: Drama, Crime
3: Drama, History, War
4: Drama
5: Animation, Family, Fantasy
6: Drama, Action, Crime, Thriller
7: Comedy, Drama, Romance
8: Fantasy, Drama, Crime
9: 
10: 
11: 
12: 
13: 
14: 
15: 
16: 
17: 
18: 
19: 


Output()

Atype : <class 'agentics.core.agentics.genre'>
genre: Drama, Crime

genre: Drama, Comedy, War

genre: Drama, Western

genre: Drama, Western, War

genre: Western, Drama, Crime

genre: Drama, Crime

genre: Drama, Action, Crime, Thriller

genre: Western, Drama, War

genre: Fantasy, Drama, Crime

genre: Drama, Crime

genre: Science Fiction, Adventure, Drama

genre: Drama, Crime

genre: Drama, Crime

genre: Drama, Fantasy

genre: Drama, Adventure

genre: Drama, War

genre: Comedy, Drama, War

genre: Drama, Crime

genre: Comedy, Drama, Thriller

genre: Drama, Crime




Atype : <class 'agentics.core.utils.AType#movie_name:genre:descriptionOptional'>
movie_name: The Shawshank Redemption
genre: Comedy, Drama, Romance
description: Imprisoned in the 1940s for the double murder of his wife and her lover,
  upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where
  he puts his accounting skills to work for an amoral warden. During his long stretch
  in prison, Dufresne comes to be admired by the other inmates -- including an older
  prisoner named Red -- for his integrity and unquenchable sense of hope.

movie_name: The Godfather
genre: Crime, Drama
description: Spanning the years 1945 to 1955, a chronicle of the fictional Italian-American
  Corleone crime family. When organized crime family patriarch, Vito Corleone barely
  survives an attempt on his life, his youngest son, Michael steps in to take care
  of the would-be killers, launching a campaign of bloody revenge.

movie_name: The Godfather Part II
genre: Animation, Drama, War
